In [46]:
from sympy import symbols, acos, diff, sin, cos, lambdify
import numpy as np
import math

In [47]:
xi,yi,zi,xj,yj,zj,xk,yk,zk,xl,yl,zl,k = symbols('xi yi zi xj yj zj xk yk zk xl yl zl k', real=True)

In [48]:
phi = ((((yj-yi)*(zk-zj)-(zj-zi)*(yk-yj))*((yk-yj)*(zl-zk)-(zk-zj)*(yl-yk)))+(((zj-zi)*(xk-xj)-(xj-xi)*(zk-zj))*((zk-zj)*(xl-xk)-(xk-xj)*(zl-zk)))
               +(((xj-xi)*(yk-yj)-(yj-yi)*(xk-xj))*((xk-xj)*(yl-yk)-(yk-yj)*(xl-xk))))/((((yj-yi)*(zk-zj)-(zj-zi)*(yk-yj))**2+((zj-zi)*(xk-xj)-(xj-xi)*(zk-zj))**2+((xj-xi)*(yk-yj)-(yj-yi)*(xk-xj))**2)**0.5*
                                                                                        (((yk-yj)*(zl-zk)-(zk-zj)*(yl-yk))**2+((zk-zj)*(xl-xk)-(xk-xj)*(zl-zk))**2+((yl-yk)-(yk-yj)*(xl-xk))**2)**0.5)

In [67]:
w1n=(((yj-yi)*(zk-zj)-(zj-zi)*(yk-yj))**2+((zj-zi)*(xk-xj)-(xj-xi)*(zk-zj))**2+((xj-xi)*(yk-yj)-(yj-yi)*(xk-xj))**2)**0.5
w2n=(((yk-yj)*(zl-zk)-(zk-zj)*(yl-yk))**2+((zk-zj)*(xl-xk)-(xk-xj)*(zl-zk))**2+((xk-xj)*(yl-yk)-(yk-yj)*(xl-xk))**2)**0.5
w1 = [((yj-yi)*(zk-zj)-(zj-zi)*(yk-yj))/w1n,((zj-zi)*(xk-xj)-(xj-xi)*(zk-zj))/w1n,((xj-xi)*(yk-yj)-(yj-yi)*(xk-xj))/w1n]
w2 = [((yk-yj)*(zl-zk)-(zk-zj)*(yl-yk))/w2n,((zk-zj)*(xl-xk)-(xk-xj)*(zl-zk))/w2n,((xk-xj)*(yl-yk)-(yk-yj)*(xl-xk))/w2n]

In [68]:
phi = acos((w1[0]*w2[0]+w1[1]*w2[1]+w1[2]*w2[2])/((w1[0]**2+w1[1]**2+w1[2]**2)**0.5*(w2[0]**2+w2[1]**2+w2[2]**2)**0.5))

In [49]:
test = lambdify((xi,yi,zi,xj,yj,zj,xk,yk,zk,xl,yl,zl),(phi.diff(xi)*phi.diff(xi)),'numpy')

In [50]:
c1 = np.array([27.34 , 24.43  , 2.614])
c2 = np.array([26.266, 25.413,  2.842])
c3 = np.array([25.112, 24.88  , 3.649])
c4 = np.array([25.353 ,24.86  , 5.134])

In [57]:
c1.sum(-1)

54.38399999999999

In [61]:
def calc_dihedral(u1, u2, u3, u4):
    """ Calculate dihedral angle method. From bioPython.PDB
    (adapted to np.array)
    Calculate the dihedral angle between 4 vectors
    representing 4 connected points. The angle is in
    [-pi, pi].
    """

    a1 = u2 - u1
    a2 = u3 - u2
    a3 = u4 - u3

    v1 = np.cross(a1, a2)
    print(v1)
    print((v1 * v1).sum(-1)**0.5)
    
    v1 = v1 / (v1 * v1).sum(-1)**0.5
    
    v2 = np.cross(a2, a3)
    v2 = v2 / (v2 * v2).sum(-1)**0.5
    porm = np.sign((v1 * a3).sum(-1))
    rad = np.arccos((v1*v2).sum(-1) / ((v1**2).sum(-1) * (v2**2).sum(-1))**0.5)
    if not porm == 0:
        rad = rad * porm

    return rad,(v1*v2).sum(-1) / ((v1**2).sum(-1) * (v2**2).sum(-1))**0.5

In [62]:
def calc_dihedral2(p1,p2,p3,p4):
    # Calculate coordinates for vectors q1, q2 and q3
    q1 = np.subtract(p2,p1) # b - a
    q2 = np.subtract(p3,p2) # c - b
    q3 = np.subtract(p4,p3) # d - c
    
    # Calculate cross vectors
    q1_x_q2 = np.cross(q1,q2)
    q2_x_q3 = np.cross(q2,q3)

    # Calculate normal vectors
    n1 = q1_x_q2/np.sqrt(np.dot(q1_x_q2,q1_x_q2))
    n2 = q2_x_q3/np.sqrt(np.dot(q2_x_q3,q2_x_q3))

    """Function to calculate orthogonal unit vectors"""
    # Calculate unit vectors
    u1 = n2
    u3 = q2/(np.sqrt(np.dot(q2,q2)))
    u2 = np.cross(u3,u1)

    # Calculate cosine and sine
    cos_theta = np.dot(n1,u1)
    sin_theta = np.dot(n1,u2)
    
    # Calculate theta
    theta = -math.atan2(sin_theta,cos_theta) # it is different from Fortran math.atan2(y,x)
    theta_deg = np.degrees(theta)
    # Show results
    return theta_deg,theta

In [63]:
print(calc_dihedral(c1,c2,c3,c4))
print(calc_dihedral2(c1,c2,c3,c4))

[0.914805 0.603606 1.706824]
2.028412324513192
(1.4026412441174954, 0.16736373976456023)
(80.3654234589115, 1.4026412441174954)


In [69]:
phi.subs(xi,c1[0]).subs(yi,c1[1]).subs(zi,c1[2]).subs(xj,c2[0]).subs(yj,c2[1]).subs(zj,c2[2]).subs(xk,c3[0]).subs(yk,c3[1]).subs(zk,c3[2]).subs(xl,c4[0]).subs(yl,c4[1]).subs(zl,c4[2])

1.40264124411749